In [22]:
import pandas as pd
import requests
import lxml.html as html
import os
import datetime
import requests
from bs4 import BeautifulSoup

In [23]:
#Teniendo la url para hacer el web scraping
urlModalidad='https://admision.unmsm.edu.pe/WebsiteExa_20232/'

def get_modalidad_url(urlModalidad):
    response = requests.get(urlModalidad, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        urls = [urlModalidad+a['href'][::] for a in soup.select('tbody tr td.text-center a')]
    return urls


In [24]:
get_modalidad_url(urlModalidad)

['https://admision.unmsm.edu.pe/WebsiteExa_20232/A.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/C.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/D.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/E.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/F.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/G.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/H.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/I.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/M.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/N.html',
 'https://admision.unmsm.edu.pe/WebsiteExa_20232/O.html']

In [25]:
def tabla_modalidad(urlModalidad):
    response = requests.get(urlModalidad, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        contents = [a.text.lower() for a in soup.select('tbody tr td.text-center a')]
        tabla =pd.DataFrame(contents,columns=['Modalidad']).reset_index()
        tabla.rename(columns={'index': 'id_modalidad'},inplace=True)
    return tabla


In [26]:
#creando la tabla de modalidades
tabla_modalidad=tabla_modalidad(urlModalidad)

In [27]:
#Exportando las modalidades de Ingreso 
tabla_modalidad.to_csv('./data/tabla_modalidad.csv',index=False,sep=',')

In [28]:
#creando la funcion para obtener la los links de cada carrera por modaldidad 
def get_carrera_url(urlAdmision):
    response = requests.get(urlAdmision, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        urls = [urlAdmision[:-7:]+a['href'][1::] for a in soup.select('tbody tr td.text-center a')]
        return urls

In [29]:
def tabla_carrera(urlAdmision):
    response = requests.get(urlAdmision, headers={'Content-Type': 'text/html; charset=utf-8'})
    if response.status_code == 200:
    #traemos el contenido de la web 
        soup = BeautifulSoup(response.content.decode('utf-8'), 'html.parser')
        # estamos creando la url de todas las areas
        tabla = [[a['href'][4:-7:],a.text.upper()] for a in soup.select('tbody tr td.text-center a')]
        tabla=pd.DataFrame(tabla,columns=['IdCarrera','Carrera'])
        return tabla

In [30]:
tabla_carrera=tabla_carrera('https://admision.unmsm.edu.pe/WebsiteExa_20232/A.html')
tabla_carrera.IdCarrera=tabla_carrera.IdCarrera.astype(int)

In [31]:
#opteniendo nuestra tabla de carreras 
tabla_carrera

,IdCarrera,Carrera
0,11,MEDICINA HUMANA
1,12,OBSTETRICIA
2,13,ENFERMERÍA
3,141,TEC. MED. LAB. CLÍNICO Y ANATOMÍA PATOLÓGICA
4,142,TEC. MED. TERAPIA FÍSICA Y REHABILITACIÓN
...,...,...
73,192,INGENIERÍA ELÉCTRICA
74,193,INGENIERÍA DE TELECOMUNICACIONES
75,194,INGENIERÍA BIOMÉDICA
76,201,INGENIERÍA DE SISTEMAS


In [32]:
#exportando nuestra tabla de carreras 
tabla_carrera.to_csv('./data/tabla_carrera.csv',sep=',',index=False)

In [33]:
url='https://admision.unmsm.edu.pe/WebsiteExa_20232/'

urlsModalidad=get_modalidad_url(url)

In [34]:
tabla_modalidad

,id_modalidad,Modalidad
0,0,educación básica regular (ebr) y educación bás...
1,1,primeros puestos de educación secundaria
2,2,traslado interno
3,3,graduados o titulados
4,4,traslado externo nacional
5,5,traslado externo internacional
6,6,deportista calificado
7,7,deportistas calificados de alto nivel
8,8,comunidades nativas
9,9,personas con discapacidad


In [35]:
#en esta funcion se introduce una lista
#devuelve el dataframe de todas las carreraspor modalidad y con id modaldiad
def get_df(urls,num):
    resultados = pd.DataFrame()
    for url in urls:
        tables = pd.read_html(url, encoding='utf-8')
        table = pd.concat(tables)
        resultados = pd.concat([resultados, table], ignore_index=True)
    resultados['IdModalidad']=num
    return resultados
    

In [36]:
#recolectamos todas las url opr modalidad de ingreso
urlsModalidad=get_modalidad_url(url)
urlsCarrera=[]
for i , urlModalidad in enumerate(urlsModalidad):
    #guardando las url de carrera
    urlsCarrera.append(get_carrera_url(urlModalidad))


In [37]:
#probando la funcion
resultados=get_df(urlsCarrera[4],2)
resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODIGO               84 non-null     int64  
 1   APELLIDOS Y NOMBRES  84 non-null     object 
 2   ESCUELA PROFESIONAL  84 non-null     object 
 3   PUNTAJE FINAL        84 non-null     object 
 4   MERITO E.P           19 non-null     float64
 5   OBSERVACI&OacuteN    41 non-null     object 
 6   IdModalidad          84 non-null     int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 4.7+ KB


In [38]:
resultados['PUNTAJE FINAL'].unique()

array([958.625, 721.75, 951.875, 913.0, 966.25, 670.5, 831.875, 1008.5,
       546.0, 564.875, 551.625, 680.625, 755.0, 802.875, 869.625, 743.125,
       820.875, 438.625, 468.5, 343.75, 'Art. 71º Reglamento', 566.0,
       776.125, 397.0, 600.625, 416.125, 871.875, 720.625, 311.375, 599.5,
       99.5, 605.375, 735.0, 543.75, 558.375, 651.625, 353.875, 254.75,
       361.75, 573.0, 567.125, 804.25, 576.375, 161.75, 536.375, 518.375,
       480.375, 438.125, 199.75, 712.75, 691.625, 649.375, 290.5, 510.75,
       452.75, 410.75, 423.75, 902.875, 384.875], dtype=object)

In [39]:
resultados['PUNTAJE FINAL']=='Art. 71º Reglamento'

0     False
1     False
2     False
3     False
4     False
      ...  
79     True
80     True
81     True
82    False
83    False
Name: PUNTAJE FINAL, Length: 84, dtype: bool

In [40]:
#creando un bucle para extraer la informacion de todos los postulantes ausentes y que ingresaron
todo = pd.DataFrame()
for i in range(len(urlsCarrera)):
    resultado=get_df(urlsCarrera[i],i)
    todo = pd.concat([todo, resultado], ignore_index=True)

In [41]:
#exportamdo los resultados de examen de admision 
#hacer un for donde extraigamos toda la data y le incluya una columna para especificar su modalidad
todo.to_csv('./data/tabla_resultados.csv',index=False,sep=',')

In [42]:
todo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25832 entries, 0 to 25831
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CODIGO               25832 non-null  int64  
 1   APELLIDOS Y NOMBRES  25832 non-null  object 
 2   ESCUELA PROFESIONAL  25832 non-null  object 
 3   PUNTAJE FINAL        25352 non-null  object 
 4   MERITO E.P           4079 non-null   float64
 5   OBSERVACI&OacuteN    4740 non-null   object 
 6   IdModalidad          25832 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1.4+ MB
